In [1]:
!pip install mlflow scikit-learn pandas numpy

  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached graphql_relay-3.2.0-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 3.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.9/753.9 kB 1.7 MB/s eta 0:00:00a 0:00:01
Using cached docker-7.1.0-py3-none-any.whl (147 kB)
Using cached graphene-3.4.3-py2.py3-none-any.whl (114 kB)
Using cached graphql_relay-3.2.0-py3-none-any.whl (16 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3━━━━━━━━━━━━━━━━━━━━  2/13 [protobuf]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [mlflow]12/13 [mlflow]skinny]]


In [2]:
import mlflow
import mlflow.sklearn

import numpy as np
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

In [3]:
# This is your local "Azure ML experiment" equivalent
mlflow.set_tracking_uri("file:./mlruns")  # local folder
mlflow.set_experiment("day3_azure_style_experiment")

print("Tracking URI:", mlflow.get_tracking_uri())
print("Experiment:", mlflow.get_experiment_by_name("day3_azure_style_experiment"))

/opt/anaconda3/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/30 22:21:25 INFO mlflow.tracking.fluent: Experiment with name 'day3_azure_style_experiment' does not exist. Creating a new experiment.


Tracking URI: file:./mlruns
Experiment: <Experiment: artifact_location=('file:///Users/eliaahadi/Library/CloudStorage/GoogleDrive-elia.ahadi@gmail.com/My '
 'Drive/Work/Career/Coding_Projects/azure_mini_projects/two_weeks_upskilling/mlruns/872455256884929174'), creation_time=1764570085152, experiment_id='872455256884929174', last_update_time=1764570085152, lifecycle_stage='active', name='day3_azure_style_experiment', tags={}>


In [4]:
data = load_breast_cancer(as_frame=True)
df = data.frame

X = df.drop(columns=["target"])
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape

((455, 30), (114, 30))

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

with mlflow.start_run(run_name="logreg_baseline"):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)

    # Log params, metrics, and model
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("max_iter", 1000)

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("roc_auc", auc)

    # Save model artifact
    mlflow.sklearn.log_model(model, artifact_path="model")

    print(f"Accuracy: {acc:.3f}")
    print(f"ROC AUC:  {auc:.3f}")

2025/11/30 22:21:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/30 22:21:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Accuracy: 0.982
ROC AUC:  0.995


In [6]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
exp = client.get_experiment_by_name("day3_azure_style_experiment")
runs = client.search_runs(exp.experiment_id)
runs_df = pd.DataFrame(
    [
        {
            "run_id": r.info.run_id,
            "run_name": r.info.run_name,
            "accuracy": r.data.metrics.get("accuracy"),
            "roc_auc": r.data.metrics.get("roc_auc"),
        }
        for r in runs
    ]
)
runs_df

,run_id,run_name,accuracy,roc_auc
0,0e03c51dfdcb4e1f9c27d49aef8bcdf5,logreg_baseline,0.982456,0.99537
